In [ ]:
# !git clone https://github.com/farhanwadia/MIE1624.git

Cloning into 'MIE1624'...
remote: Enumerating objects: 21, done.
remote: Counting objects: 100% (21/21), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 21 (delta 4), reused 7 (delta 1), pack-reused 0
Unpacking objects: 100% (21/21), 91.54 KiB | 2.69 MiB/s, done.


In [ ]:
# %cd MIE1624
# %cd 'Course Presentation'

/content/MIE1624
/content/MIE1624/Course Presentation


In [ ]:
# !ls

'In-class presentation assignment - Group 14.pdf'   RSS.ipynb


In [ ]:
!pip install feedparser

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.1/81.1 KB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6066 sha256=d0d224814116b970d4ac08cad3be742ae0dee6421711ad7144c2c31bd3d0fa81
  Stored in directory: /root/.cache/pip/wheels/83/63/2f/117884c3b19d46b64d3d61690333aa80c88dc14050e269c546
Successfully built sgmllib3k


A list of all RSS feeds from the New York Times can be accessed at https://www.nytimes.com/rss.

Let's use the World feed from https://rss.nytimes.com/services/xml/rss/nyt/World.xml as an example:

In [ ]:
import feedparser

d = feedparser.parse('https://rss.nytimes.com/services/xml/rss/nyt/World.xml')

In [ ]:
# Get a list of all possible fields from the RSS
all_fields = []
for field in d.entries[0]:
    all_fields.append(field)

print(all_fields)

['title', 'title_detail', 'links', 'link', 'id', 'guidislink', 'summary', 'summary_detail', 'authors', 'author', 'author_detail', 'published', 'published_parsed', 'media_content', 'media_credit', 'credit']


In [ ]:
# Define the fields of interest that we want to obtain from the RSS
fields = ['title', 'published', 'summary', 'author', 'link']

In [ ]:
import pandas as pd

# Create a list of lists to hold the required RSS data from each entry
data = []
for i, entry in enumerate(d.entries):
    row = []
    for field in fields:
        row.append(d.entries[i][field])
    data.append(row)

# Convert the list of lists to a df
df = pd.DataFrame(data, columns = fields)

In [ ]:
df.head()

,title,published,summary,author,link
0,Live Updates: Top U.S. and Russian Diplomats M...,"Thu, 02 Mar 2023 17:31:51 +0000",The American secretary of state and the Russia...,The New York Times,https://www.nytimes.com/live/2023/03/02/world/...
1,"A Ukrainian Mayor Disappeared, but Questions o...","Thu, 02 Mar 2023 11:55:50 +0000",Traitor or hero? A decision by the former mayo...,Michael Schwirtz and Ivor Prickett,https://www.nytimes.com/2023/03/02/world/europ...
2,Anger Grows in Greece After Train Crash Kills ...,"Thu, 02 Mar 2023 17:22:08 +0000","As the death toll rose to 57, protesters clash...","Emma Bubola, Iliana Magra and Niki Kitsantonis",https://www.nytimes.com/2023/03/02/world/europ...
3,Canadians Fume as Migrants Surge at Their Border,"Thu, 02 Mar 2023 12:20:27 +0000",A surge in illegal crossings from the United S...,Norimitsu Onishi,https://www.nytimes.com/2023/03/02/world/canad...
4,"As France Faces Backlash in Africa, Macron Tri...","Thu, 02 Mar 2023 15:31:22 +0000",President Emmanuel Macron has embarked on a fo...,Roger Cohen,https://www.nytimes.com/2023/03/02/world/afric...


In [ ]:
print("The shape of the dataframe is", df.shape)

The shape of the dataframe is (59, 5)


In [ ]:
#To-do: obtain an API key from NYT to scrape the articles to get their fulltext

In [ ]:
import requests
from bs4 import BeautifulSoup

links = df["link"]

URL = links[0]
page = requests.get(URL)

soup = BeautifulSoup(page.content, "html.parser")

In [ ]:
import requests
from bs4 import BeautifulSoup

# https://proxiesapi-com.medium.com/scraping-the-new-york-times-with-python-and-beautiful-soup-6e5f3bc58e39
headers = {'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2) AppleWebKit/601.3.9 (KHTML, like Gecko) Version/9.0.2 Safari/601.3.9'}
URL = links[0]
response = requests.get(URL, headers=headers)

soup = BeautifulSoup(response.content, "html.parser")
soup



<html><head><title>nytimes.com</title><style>#cmsg{animation: A 1.5s;}@keyframes A{0%{opacity:0;}99%{opacity:0;}100%{opacity:1;}}</style></head><body style="margin:0"><p id="cmsg">Please enable JS and disable any ad blocker</p><script data-cfasync="false">var dd={'rt':'c','cid':'AHrlqAAAAAMAQ2fiYQQ5e-0AIkn2OA==','hsh':'499AE34129FA4E4FABC31582C3075D','t':'fe','s':17439,'e':'21a5f1236abf529315e88fa86146e0937a3547cc8f28055d8318b5ce915e69f5','host':'geo.captcha-delivery.com'}</script><script data-cfasync="false" src="https://ct.captcha-delivery.com/c.js"></script></body></html>

In [ ]:
page.content

b'<html><head><title>nytimes.com</title><style>#cmsg{animation: A 1.5s;}@keyframes A{0%{opacity:0;}99%{opacity:0;}100%{opacity:1;}}</style></head><body style="margin:0"><p id="cmsg">Please enable JS and disable any ad blocker</p><script data-cfasync="false">var dd={\'rt\':\'c\',\'cid\':\'AHrlqAAAAAMA2HIPuncDiQwAIkn2OA==\',\'hsh\':\'499AE34129FA4E4FABC31582C3075D\',\'t\':\'bv\',\'s\':17439,\'e\':\'72278b859eae44b7b69e189be47a29452c61a6e5a833090f6b7b4e5c08578d9a\',\'host\':\'geo.captcha-delivery.com\'}</script><script data-cfasync="false" src="https://ct.captcha-delivery.com/c.js"></script></body></html>\n'

In [ ]:
# Obtain fulltext for each article

for link in links:
    print(link)

https://www.nytimes.com/live/2023/03/02/world/russia-ukraine-news
https://www.nytimes.com/2023/03/02/world/europe/ukraine-kherson-mayor-russia.html
https://www.nytimes.com/2023/03/02/world/europe/greece-train-crash.html
https://www.nytimes.com/2023/03/02/world/canada/canada-migrants-immigration.html
https://www.nytimes.com/2023/03/02/world/africa/france-macron-africa.html
https://www.nytimes.com/2023/03/02/world/middleeast/tunisia-president-migrants-africa.html
https://www.nytimes.com/2023/03/02/world/europe/manchester-arena-bombing-report-ariana-grande.html
https://www.nytimes.com/2023/03/01/world/europe/ukraine-russia-tanks.html
https://www.nytimes.com/2023/03/01/world/africa/nigeria-president-election-tinubu.html
https://www.nytimes.com/2023/03/02/world/europe/harry-meghan-frogmore-cottage-uk-home.html
https://www.nytimes.com/article/nigeria-election.html
https://www.nytimes.com/2023/03/01/world/middleeast/iran-schoolgirls-poison.html
https://www.nytimes.com/2023/03/01/world/europe/

**Extract most commonly used elements in RSS feeds such as “title”,
“link”, “description”, “publication date”, “entry ID”, etc. Follow “links” in the RSS feed and
extract text of news articles using web scraping in Python or other Python methods. You can use
BeautifulSoup library in Python for pulling data out of HTML and XML files. In your IPython
example, use a news web-site of your choice and create a Pandas dataframe from your RSS parsing
results representing fields of metadata from that article, e.g., title, date, link, author, etc., as well
as text of the news article. Print a table with your results and export those as a csv file with each
row as an article and columns as article’s metadata (including full text of each article).**

### Adding full texts for the corresponding articles,

In [ ]:
!pip install newspaper3k

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.1/211.1 KB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.9/93.9 KB 9.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 53.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for tinysegmenter: filename=tinysegmenter-0.3-py3-none-any.whl size=13552 sha256=a0f6a97221005710d2998a0e86fd6fc7b5da28216493131f4df1f713b5f9c793
  Stored in directory: /root/.cache/pip/wheels/99/74/83/8fac1c8d9c648cfabebbbffe97a889f6624817f3aa0bbe6c09
  Created wheel for feedfinder2: filename=feedfinder2-0.0.4-py3-none-any.whl size=3354 sha256=a8625db33ea454c92d2f95ab643aceae1b439c1092210d06cb80db66a696373d
  Stored in directory: /root/.cache/pip/wheels/b6/09/68/a9f15498ac02c23dde29f18745bc6a6f574ba4ab41861a3575
  C

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
from newspaper import Article
article_text_dict = {}
for link in links:
  article = Article(link)
  article.download()
  article.parse()
  article.nlp()
  article_text_dict[link] = article.text

In [ ]:
article_text_dict

{'https://www.nytimes.com/live/2023/03/02/world/russia-ukraine-news': 'Secretary of State Antony J. Blinken met briefly on Thursday with his Russian counterpart, Sergey V. Lavrov, on the sidelines of a Group of 20 meeting, the first one-on-one meeting between the two top diplomats since Russia began its full-scale invasion of Ukraine more than a year ago.\n\nThe unexpected encounter came as the Russian government used the meeting of G20 foreign ministers in New Delhi, the Indian capital, accused Western nations of “blackmail and threats.”\n\nIn their meeting, Mr. Blinken made three points to Mr. Lavrov, a senior State Department official told reporters: the United States would continue to support Ukraine in its defense against Russia “for as long as it takes”; Russia should rejoin the New START nuclear arms control treaty from which it recently withdrew and abide by the terms; and that Russia should release Paul Whelan, an American citizen whom the U.S. says is wrongfully imprisoned.\n

In [ ]:
df['text'] = list(article_text_dict.values())
df.head()

,title,published,summary,author,link,text
0,Live Updates: Top U.S. and Russian Diplomats M...,"Thu, 02 Mar 2023 17:31:51 +0000",The American secretary of state and the Russia...,The New York Times,https://www.nytimes.com/live/2023/03/02/world/...,Secretary of State Antony J. Blinken met brief...
1,"A Ukrainian Mayor Disappeared, but Questions o...","Thu, 02 Mar 2023 11:55:50 +0000",Traitor or hero? A decision by the former mayo...,Michael Schwirtz and Ivor Prickett,https://www.nytimes.com/2023/03/02/world/europ...,"KHERSON, Ukraine — The jailhouse near the Dnip..."
2,Anger Grows in Greece After Train Crash Kills ...,"Thu, 02 Mar 2023 17:22:08 +0000","As the death toll rose to 57, protesters clash...","Emma Bubola, Iliana Magra and Niki Kitsantonis",https://www.nytimes.com/2023/03/02/world/europ...,"The station manager in Larissa, about 12 miles..."
3,Canadians Fume as Migrants Surge at Their Border,"Thu, 02 Mar 2023 12:20:27 +0000",A surge in illegal crossings from the United S...,Norimitsu Onishi,https://www.nytimes.com/2023/03/02/world/canad...,“The fact that Biden is getting tougher reinfo...
4,"As France Faces Backlash in Africa, Macron Tri...","Thu, 02 Mar 2023 15:31:22 +0000",President Emmanuel Macron has embarked on a fo...,Roger Cohen,https://www.nytimes.com/2023/03/02/world/afric...,President Emmanuel Macron of France has embark...
